In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import random
from typing import Hashable, Mapping, Dict

import numpy as np
from PIL import Image
from matplotlib import pyplot  as plt
# 打印并修改当前工作目录
print(os.getcwd())  # 检查当前工作目录
os.chdir('/home/tanzl/code/githubdemo/SyntheticTumors/TumorGenerated')
print(os.getcwd())  #

from copy import deepcopy

# 导入模块
from TumorGenerated_2d import TumorGenerated
from tqdm import tqdm
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/tanzl/code/githubdemo/SyntheticTumors/TumorGenerated
/home/tanzl/code/githubdemo/SyntheticTumors/TumorGenerated


In [2]:
input_folder = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train"
output_folder = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train_augmented"
mask_folder = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train_augmented_mask"
transform = TumorGenerated(keys=['image', 'label'],
                           prob=1,                                # 进行数据增强的概率
                           tumor_prob = [0.2, 0.2, 0.2, 0.2, 0.2])  # ['tiny'(4), 'small'(8), 'medium'(16), 'large'(32), 'mix']不同大小肿瘤的概率,括号内为半径，mix是不同大小的都有




All predefined texture have generated.


In [4]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(mask_folder):
    os.makedirs(mask_folder)

count = 0
error_count = 0
error_names = []

for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith(".png"):
        try:
            filepath = os.path.join(input_folder, filename)
            image = np.array(Image.open(filepath).convert('L'))  # Convert to grayscale

            if np.sum(image) == 0:
                continue  # Skip empty images

            label = deepcopy(image)  # Assuming a dummy label for simplicity
            image = image/255
            label[label > 0] = 1

            data = {'image': image, 'label': label}
            transformed_data = transform(data)
            
            transformed_image = Image.fromarray((transformed_data['image'] * 255).astype(np.uint8))
            transformed_image.save(os.path.join(output_folder, filename.replace(".png", "_augmented.png")))
            if np.sum(transformed_data['label']) == np.sum(label):
                print(f"No augmentation: {filename}")
                transformed_label = Image.fromarray((np.zeros_like(transformed_data['label'])).astype(np.uint8))
                transformed_label.save(os.path.join(mask_folder, filename.replace(".png", "_augmented_mask.png")))
            else:
                transformed_label = Image.fromarray((transformed_data['label'] * 255).astype(np.uint8))
                transformed_label.save(os.path.join(mask_folder, filename.replace(".png", "_augmented_mask.png")))
        except Exception as e:
            # print(f"Error: {filename}")
            error_names.append(filename)
        # print(f"Processed and saved: {filename}")

        # count += 1
        # if count > 10:
        #     break

  5%|▍         | 9387/204800 [02:25<42:46, 76.15it/s]  

No augmentation: 00191_slice_026.png


 10%|▉         | 19541/204800 [04:51<39:06, 78.96it/s]  

No augmentation: 00101_slice_015.png


 19%|█▊        | 37903/204800 [09:17<46:12, 60.20it/s]  

...

 84%|████████▍ | 171852/204800 [41:10<11:11, 49.05it/s]

No augmentation: 00355_slice_012.png


 92%|█████████▏| 188919/204800 [45:15<03:51, 68.62it/s] 

No augmentation: 00291_slice_015.png


100%|██████████| 204800/204800 [51:56<00:00, 65.71it/s] 


# 展示前后差异

In [ ]:
import os
import numpy as np
from PIL import Image

# 配置各个目录路径


original_dir = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/toy"
enhanced_dir = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/toy_augmented"
difference_dir = "/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/toy_difference"
os.makedirs(difference_dir, exist_ok=True)

def load_image_as_numpy(image_path):
    """加载图像并转换为 numpy 数组。"""
    image = Image.open(image_path).convert('L')  # 转换为灰度图像
    image = np.array(image)
    return image

def save_image_from_numpy(image, save_path):
    """将 numpy 数组保存为图像。"""
    image = Image.fromarray(image.astype(np.uint8))
    image.save(save_path)

# 获取所有原始图像文件
image_files = [f for f in os.listdir(enhanced_dir) if f.endswith(".png")]

for image_file in image_files:
    original_image_path = os.path.join(original_dir, image_file)
    enhanced_image_path = os.path.join(enhanced_dir, image_file)
    
    if not os.path.exists(enhanced_image_path):
        print(f"Enhanced image for {image_file} not found, skipping.")
        continue
    
    # 加载原始图像和增强图像
    original_image = load_image_as_numpy(original_image_path)
    enhanced_image = load_image_as_numpy(enhanced_image_path)
    
    # 确保图像尺寸一致
    if original_image.shape != enhanced_image.shape:
        print(f"Image shapes do not match for {image_file}, skipping.")
        continue
    
    # 计算差异
    difference_image = np.abs(enhanced_image - original_image)
    if np.sum(difference_image) > 0:
        print(image_file)
    
    # 保存差异图像
    difference_image_path = os.path.join(difference_dir, image_file)
    save_image_from_numpy(difference_image, difference_image_path)

print("Difference computation and saving completed.")
